***

<center><h1>Transcript Data Frame Builder Notebook <center><h1>

***

## Goals of this notebook:
    
* Connect to one of our local council's database and filestore.
* Retrieve all available meeting transcripts including their meta data for the corresponding council.
* Build a dataframe from all of this for later analysis.

## Load relevant libraries for the task

In [40]:
## For connecting to the database and filestore
import fireo
from google.auth.credentials import AnonymousCredentials
from google.cloud.firestore import Client
from gcsfs import GCSFileSystem

## For calling cdp database and pipeline functions
from cdp_backend.database import models as db_models
#from cdp_backend.pipeline.transcript_model import Transcript

## For working with the data and our local file system
import pandas as pd
import os 

## Connect to Council's Data Base

In [11]:
## For Seattle
fireo.connection(client=Client(
    project="cdp-seattle-staging-dbengvtn",
    credentials=AnonymousCredentials()
))

## Connect to Council's File Store

In [12]:
## For Seattle
fs = GCSFileSystem(project="cdp-seattle-staging-dbengvtn", token="anon")

In [13]:
## Fetch details for all available transcripts for council of interest
transcript_model = list(db_models.Transcript.collection.fetch())[0:]

In [14]:
## Total transcripts for this particular council at this time
len(transcript_model)

58

## Dowload all transcripts to local machine
__Note:__ _The function below is for downloading these transcripts to your local machine and therefore may be time and space intensive._

In [21]:
## Finish documenting function

## Function to Download all transcripts locally
def download_transcripts(list_of_transcripts_db_ref, council):
    '''
    Takes a list of database objects, and a chosen council as arguments.
    council argument must be in string format.
    Creates a file for transcripts to download to.
    Downloads database objects to local machine as .json type.
    Transcript files are saved with name incrementally counting up from 0.
    '''
    num = 0  ## For adding index to file names
    for transcripts in range(0, len(list_of_transcripts_db_ref)):
        s = str(num)
        fs.get(transcript_model[0].file_ref.get().uri, 
               "transcripts/"+council+"/"+council+"-transcript"+s+".json")
        del transcript_model[0]
        num = num + 1
    print("Transcript download complete!")

In [22]:
download_transcripts(transcript_model, council="seattle")

### Now that we have the transcripts downloaded we can read one into a pandas dataframe:

In [33]:
transcript_df = pd.read_json("transcripts/seattle/seattle-transcript1.json")
transcript_df.sample(5)

,generator,confidence,session_datetime,created_datetime,sentences,annotations
372,CDP WebVTT Conversion -- CDP v3.0.3,0.97,2022-01-26T09:30:00-08:00,2022-01-27T01:39:07.147398,"{'index': 372, 'confidence': 0.97, 'start_time...",NaN
271,CDP WebVTT Conversion -- CDP v3.0.3,0.97,2022-01-26T09:30:00-08:00,2022-01-27T01:39:07.147398,"{'index': 271, 'confidence': 0.97, 'start_time...",NaN
430,CDP WebVTT Conversion -- CDP v3.0.3,0.97,2022-01-26T09:30:00-08:00,2022-01-27T01:39:07.147398,"{'index': 430, 'confidence': 0.97, 'start_time...",NaN
62,CDP WebVTT Conversion -- CDP v3.0.3,0.97,2022-01-26T09:30:00-08:00,2022-01-27T01:39:07.147398,"{'index': 62, 'confidence': 0.97, 'start_time'...",NaN
15,CDP WebVTT Conversion -- CDP v3.0.3,0.97,2022-01-26T09:30:00-08:00,2022-01-27T01:39:07.147398,"{'index': 15, 'confidence': 0.97, 'start_time'...",NaN


### We can see some interesting data here, but to get the actual text data we must "flatten" the nested objects out in the "sentences" column.

In [34]:
df_flattened_sentences = pd.json_normalize(transcript_df['sentences'])
df_flattened_sentences.sample(5)

,index,confidence,start_time,end_time,words,text,speaker_index,speaker_name,annotations
422,422,0.97,3286.016,3288.351,"[{'index': 0, 'start_time': 3286.016, 'end_tim...",We are still on.,81,None,None
205,205,0.97,1509.808,1520.952,"[{'index': 0, 'start_time': 1517.415, 'end_tim...",The tribe has been a great partner for us thro...,25,None,None
14,14,0.97,61.094,69.035,"[{'index': 0, 'start_time': 66.699, 'end_time'...",If there is no objection the agenda will be ad...,6,None,None
79,79,0.97,611.143,614.914,"[{'index': 0, 'start_time': 613.746, 'end_time...",Because public safety is a core foundation of ...,6,None,None
255,255,0.97,1793.591,1797.228,"[{'index': 0, 'start_time': 1793.591, 'end_tim...",Great.,43,None,None


### That looks better! But we lost some data along the way, so lets create a DataFrame with the rest of the important data so we can merge them.

In [38]:
transcript_df = pd.concat([transcript_df, df_flattened_sentences], axis=1)
transcript_df.sample(5)

,generator,confidence,session_datetime,created_datetime,sentences,annotations,index,confidence,start_time,end_time,...,annotations,index,confidence,start_time,end_time,words,text,speaker_index,speaker_name,annotations
315,CDP WebVTT Conversion -- CDP v3.0.3,0.97,2022-01-26T09:30:00-08:00,2022-01-27T01:39:07.147398,"{'index': 315, 'confidence': 0.97, 'start_time...",NaN,315,0.97,2493.457,2505.870,...,None,315,0.97,2493.457,2505.870,"[{'index': 0, 'start_time': 2504.201, 'end_tim...",So we are currently starting to develop and th...,47,None,None
129,CDP WebVTT Conversion -- CDP v3.0.3,0.97,2022-01-26T09:30:00-08:00,2022-01-27T01:39:07.147398,"{'index': 129, 'confidence': 0.97, 'start_time...",NaN,129,0.97,928.360,933.632,...,None,129,0.97,928.360,933.632,"[{'index': 0, 'start_time': 930.763, 'end_time...","Okay, and I think Chris was going to bring the...",12,None,None
377,CDP WebVTT Conversion -- CDP v3.0.3,0.97,2022-01-26T09:30:00-08:00,2022-01-27T01:39:07.147398,"{'index': 377, 'confidence': 0.97, 'start_time...",NaN,377,0.97,3085.882,3092.155,...,None,377,0.97,3085.882,3092.155,"[{'index': 0, 'start_time': 3090.253, 'end_tim...","But I always wonder if you have one of these, ...",56,None,None
432,CDP WebVTT Conversion -- CDP v3.0.3,0.97,2022-01-26T09:30:00-08:00,2022-01-27T01:39:07.147398,"{'index': 432, 'confidence': 0.97, 'start_time...",NaN,432,0.97,3346.910,3359.556,...,None,432,0.97,3346.910,3359.556,"[{'index': 0, 'start_time': 3356.82, 'end_time...",Perhaps that's something that Oir can help us ...,82,None,None
69,CDP WebVTT Conversion -- CDP v3.0.3,0.97,2022-01-26T09:30:00-08:00,2022-01-27T01:39:07.147398,"{'index': 69, 'confidence': 0.97, 'start_time'...",NaN,69,0.97,495.895,505.238,...,None,69,0.97,495.895,505.238,"[{'index': 0, 'start_time': 505.071, 'end_time...",And also policy initiatives like Digital equit...,6,None,None


***

## Building the Transcript DataFrame

* Okay, we have:
* Connected to a council's database and filestore.
* Created folders on our local machine to house transcripts
* Downloaded all available transcripts corresponding to our chosen council.
* Finally we have read one into a pandas dataframe.
* Now let's build a DataFrame of all the transcripts we have downloaded so we can have something fun to analyze.

***

In [42]:
## File containing all of our transcripts
file = ("transcripts/seattle/")

## Create a list of all transcript files inside our file
path = file
transcript_list = list(os.listdir(path))

## See how many transcripts we have in our local folder currently
## Note: This number should match our total of database objects from earlier
print("Total Transcripts: ", len(transcript_list))

Total Transcripts:  58


In [48]:
def read_in_transcripts(file, council):
    '''
    Takes a file of .json objects, and a chosen council as arguments.
    Council argument must be in string format.
    Reads .json files into pandas dataframe.
    Flattens out nested text in the "sentences" column.
    Returns a pandas dataframe object made of all transcript
    dataframe objects concantenated.
    '''
    num = 0
    transcript_df = pd.DataFrame()
    for transcripts in range(0, len(transcript_list)):
        s = str(num)
        text_df = pd.read_json(file+council+"-transcript"+s+".json")
        df_flattened_sentences = pd.json_normalize(text_df['sentences'])
        df = pd.concat([text_df, df_flattened_sentences], axis=1)        
        transcript_df = transcript_df.append(df)
        num = num + 1
    return(transcript_df)

In [49]:
transcript_df = read_in_transcripts(file, council="seattle")

In [50]:
transcript_df.sample(5)

,generator,confidence,session_datetime,created_datetime,sentences,annotations,index,confidence,start_time,end_time,words,text,speaker_index,speaker_name,annotations
213,CDP WebVTT Conversion -- CDP v3.0.2,0.97,2021-11-10T09:00:00-08:00,2022-01-12T00:21:15.712232,"{'index': 213, 'confidence': 0.97, 'start_time...",NaN,213,0.97,1534.933,1548.580,"[{'index': 0, 'start_time': 1544.109, 'end_tim...",I'm just wondering--I know we had a really rob...,7,None,None
507,CDP WebVTT Conversion -- CDP v3.0.2,0.97,2021-12-07T14:00:00-08:00,2022-01-11T23:22:59.273131,"{'index': 507, 'confidence': 0.97, 'start_time...",NaN,507,0.97,3532.795,3533.396,"[{'index': 0, 'start_time': 3532.795, 'end_tim...",They do?,119,None,None
132,CDP WebVTT Conversion -- CDP v3.0.5,0.97,2022-02-16T09:30:00-08:00,2022-02-17T01:37:36.262741,"{'index': 132, 'confidence': 0.97, 'start_time...",NaN,132,0.97,949.448,953.319,"[{'index': 0, 'start_time': 949.448, 'end_time...",It has been moved and Seconded.,44,None,None
1171,CDP WebVTT Conversion -- CDP v3.0.2,0.97,2021-11-10T17:30:00-08:00,2022-01-12T00:42:13.458424,"{'index': 1171, 'confidence': 0.97, 'start_tim...",NaN,1171,0.97,10640.296,10643.199,"[{'index': 0, 'start_time': 10641.564, 'end_ti...",We spend all day with these kids who look up t...,248,None,None
1426,CDP WebVTT Conversion -- CDP v3.0.2,0.97,2021-11-10T09:00:00-08:00,2022-01-12T00:21:15.712232,"{'index': 1426, 'confidence': 0.97, 'start_tim...",NaN,1426,0.97,9846.736,9860.517,"[{'index': 0, 'start_time': 9858.515, 'end_tim...",Sdot 3 B 1 sponsored by Council member Strauss...,174,None,None


***

<center><h2>Exporting DataFrame<center><h2>
    
***

* Now that we have created a dataframe full of all available transcripts for our council, we can export this dataframe for analysis at a later date.
* In this example we will use .csv format.

In [51]:
transcript_df.to_csv('seattle-transcripts.csv', index = False)

* Let's make sure it worked by reading our new .csv file into a pandas dataframe

In [53]:
df = pd.read_csv("seattle-transcripts.csv")
df.sample(5)

,generator,confidence,session_datetime,created_datetime,sentences,annotations,index,confidence.1,start_time,end_time,words,text,speaker_index,speaker_name,annotations.1
29308,CDP WebVTT Conversion -- CDP v3.0.2,0.97,2021-12-06T09:30:00-08:00,2022-01-11T23:23:15.636624,"{'index': 228, 'confidence': 0.97, 'start_time...",NaN,228,0.97,1681.579,1698.863,"[{'index': 0, 'start_time': 1697.796, 'end_tim...","District 4 this past week, my staff attended t...",23.0,NaN,NaN
45547,CDP WebVTT Conversion -- CDP v3.0.3,0.97,2022-02-02T14:00:00-08:00,2022-02-03T01:34:01.086840,"{'index': 326, 'confidence': 0.97, 'start_time...",NaN,326,0.97,2363.828,2375.239,"[{'index': 0, 'start_time': 2374.538, 'end_tim...",I didn't really hear any potential issues get ...,61.0,NaN,NaN
3539,CDP WebVTT Conversion -- CDP v3.0.2,0.97,2021-12-01T14:00:00-08:00,2022-01-11T23:19:53.752622,"{'index': 237, 'confidence': 0.97, 'start_time...",NaN,237,0.97,2061.893,2069.133,"[{'index': 0, 'start_time': 2067.899, 'end_tim...",Because I understand the need to have routine ...,50.0,NaN,NaN
29003,CDP WebVTT Conversion -- CDP v3.0.3,0.97,2022-02-01T09:30:00-08:00,2022-02-02T06:42:03.012222,"{'index': 1277, 'confidence': 0.97, 'start_tim...",NaN,1277,0.97,8808.499,8816.607,"[{'index': 0, 'start_time': 8816.34, 'end_time...",But having your list of issues gives us a grea...,136.0,NaN,NaN
33367,CDP WebVTT Conversion -- CDP v3.0.2,0.97,2021-12-13T14:00:00-08:00,2022-01-11T22:45:32.348937,"{'index': 1504, 'confidence': 0.97, 'start_tim...",NaN,1504,0.97,10545.167,10570.126,"[{'index': 0, 'start_time': 10559.682, 'end_ti...",We are building in a touch point where the pub...,348.0,NaN,NaN


* Okay we have successfully constructed a dataframe of insteresting transcript data for analysis.
* This is admittedly a raw dataframe that needs some more work depending on the types of analysis being performed.
* It is important to note that by changing our connection to a different council's database and filestore, we can create the same dataframe for whatever council we choose.
* For example if we just changed the code cells at the beginning of this notebook to connect to King County WA, we could reuse the entire notebook by changing the arguments in our functions.

* Example:

In [54]:
## For connecting to King County WA database
fireo.connection(client=Client(
    project="cdp-king-county-b656c71b",
    credentials=AnonymousCredentials()
))

In [55]:
## For connecting to King County WA filestore
fs = GCSFileSystem(project="cdp-king-county-b656c71b", token="anon")

* These two code cells are the same cells from the beginning of this notebook with the council changed from Seattle WA, to King County WA. 
* With simply swapping the council's project id out we can repeat the notebook for King County WA now.
* Just don't forget to change our arguments when calling the functions like our "download_transcripts" function:
```python
download_transcripts(transcript_model, council="king-county")
```